#### Pacotes

In [1]:
import pandas as pd
import numpy as np

## Exemplo curva ABC

In [2]:
data = {
    'pais':['BRASIL','BRASIL','BRASIL', 'PERU','PERU','PERU'],
    'cod_produto':['1','2','3','10','20','30'],
    'receita': [5085,100,5655,451,456,458]
}

_df = pd.DataFrame(data)
_df

,pais,cod_produto,receita
0,BRASIL,1,5085
1,BRASIL,2,100
2,BRASIL,3,5655
3,PERU,10,451
4,PERU,20,456
5,PERU,30,458


#### Ranqueando por receita

In [3]:
_df = _df.groupby(['pais', 'cod_produto'], as_index=False).agg({'receita':sum}) # quando os dados estiverem mais granulares.
_df = _df.sort_values(['pais', 'receita'], ascending=False,)
_df

,pais,cod_produto,receita
5,PERU,30,458
4,PERU,20,456
3,PERU,10,451
2,BRASIL,3,5655
0,BRASIL,1,5085
1,BRASIL,2,100


#### Fanzendo o share por pais

In [4]:
_df_sum = _df.groupby(['pais'], as_index=False).agg({'receita':sum}).rename(columns={'receita':'receita_sum'})

_df = _df.merge(_df_sum, on=['pais'], how= 'left')

_df['share'] = np.where(_df['receita_sum'] == 0, 0,_df['receita']/_df['receita_sum'])

_df

,pais,cod_produto,receita,receita_sum,share
0,PERU,30,458,1365,0.335531
1,PERU,20,456,1365,0.334066
2,PERU,10,451,1365,0.330403
3,BRASIL,3,5655,10840,0.521679
4,BRASIL,1,5085,10840,0.469096
5,BRASIL,2,100,10840,0.009225


#### Fazendo a soma acumulativa (cumsum)

In [5]:
_df['cumsum'] = _df.groupby(['pais']).agg({'share':'cumsum'})
_df

,pais,cod_produto,receita,receita_sum,share,cumsum
0,PERU,30,458,1365,0.335531,0.335531
1,PERU,20,456,1365,0.334066,0.669597
2,PERU,10,451,1365,0.330403,1.000000
3,BRASIL,3,5655,10840,0.521679,0.521679
4,BRASIL,1,5085,10840,0.469096,0.990775
5,BRASIL,2,100,10840,0.009225,1.000000


#### Regra Curva ABC

In [6]:
"""
Regra 
Se CumSum(Valor Acumudado do share) <= 80% classe_abc = A 
Se CumSum(Valor Acumudado do share) > 80% e CumSum(Valor Acumudado do share) <= A80% + B15% classe_abc = B
Se não classe_abc = C
"""

_df['classe_abc'] = np.where(_df['cumsum'] <= 0.60, 'A', 
                    np.where( (_df['cumsum'] > 0.60 ) & (_df['cumsum'] <= 0.80), 'B', 'C'))

_df

,pais,cod_produto,receita,receita_sum,share,cumsum,classe_abc
0,PERU,30,458,1365,0.335531,0.335531,A
1,PERU,20,456,1365,0.334066,0.669597,B
2,PERU,10,451,1365,0.330403,1.000000,C
3,BRASIL,3,5655,10840,0.521679,0.521679,A
4,BRASIL,1,5085,10840,0.469096,0.990775,C
5,BRASIL,2,100,10840,0.009225,1.000000,C


In [12]:
def highlight_cols(s):
    if s == 'A':
        color = 'green'
    elif s == 'B':
        color = 'blue'
    else:
        color = 'red'
    return 'background-color: % s' % color

In [14]:
_df.style.format( '{:.2f}', subset=['share', 'cumsum'] ) \
    .background_gradient(subset=['share', 'cumsum'] , cmap='Reds')\
    .applymap(highlight_cols, subset =['classe_abc'])
    # .applymap(lambda x: 'color:black;', subset =['classe_abc'])

,pais,cod_produto,receita,receita_sum,share,cumsum,classe_abc
0,PERU,30,458,1365,0.34,0.34,A
1,PERU,20,456,1365,0.33,0.67,B
2,PERU,10,451,1365,0.33,1.00,C
3,BRASIL,3,5655,10840,0.52,0.52,A
4,BRASIL,1,5085,10840,0.47,0.99,C
5,BRASIL,2,100,10840,0.01,1.00,C
